asd

In [1]:
#pip uninstall pyannote.audio

In [2]:
#!pip install pyannote.audio

#from pydub import AudioSegment
#from pyannote.audio import Pipeline

In [3]:
try:
    import pyannote.audio
    print("pyannote.audio is installed.")
except ImportError:
    print("pyannote.audio is not installed.")

pyannote.audio is installed.


In [4]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token="")
#from collections import defaultdict
#from pyannote.audio import Pipeline
#pipeline = Pipeline.from_pretrained(
  #  "pyannote/speaker-diarization-3.1",
   # use_auth_token="")

# send pipeline to GPU (when available)
#import torch
#pipeline.to(torch.device("cuda"))




SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /pyannote/speaker-diarization-3.1/resolve/main/config.yaml (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))"), '(Request ID: 45c554eb-614a-4828-8d0e-aeaf7c29d361)')

In [ ]:
# apply pretrained pipeline
diarization = pipeline("TrumpHarrisDebate.mp3")


NameError: name 'pipeline' is not defined

In [ ]:
# print the result
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")